# Imports

In [50]:
import os
import numpy
import pickle

In [51]:
%reload_ext autoreload
%autoreload 2
import face_utilities as face

PHASE 1 - Parse the face dataset

In [52]:
print('Parsing Face Dataset STARTED...')
images, labels = face.parse_face_dataset()
print('Length:', len(images), '|', len(labels))
print('Parsing Face Dataset COMPLETE!')

Parsing Face Dataset STARTED...
Total Face Images Found: 558
Length: 558 | 558
Parsing Face Dataset COMPLETE!


PHASE 2 - Perform face detection

In [53]:
print('Face Image Preprocessing STARTED...')
cropped_images, cropped_labels = face.detect_faces(zip(images, labels))
print('Length:', len(cropped_images), '|', len(cropped_labels))
print('Face Image Preprocessing COMPLETE!')

print('Saving...')
pickle.dump(cropped_images, open(os.path.join('Face_Output', 'cropped_faces.pickle'), 'wb'))
numpy.save(os.path.join('Face_Output', 'cropped_labels'), cropped_labels)
print('Done!')

Face Image Preprocessing STARTED...
Length: 490 | 490
Face Image Preprocessing COMPLETE!
Saving...
Done!


PHASE 3 - Perform facial feature extraction

In [49]:
print("Face Feature Extraction STARTED...")
features, feature_labels = face.extract_features(zip(cropped_images, labels))
print('Length:', len(features), '|', len(feature_labels))
print("Face Feature Extraction COMPLETE!")

print('Saving feature array...')
numpy.save(os.path.join('Face_Output', 'features'), features)
numpy.save(os.path.join('Face_Output', 'feature_labels'), feature_labels)
print('Done!')

Face Feature Extraction STARTED...
Length: 334 | 334
Face Feature Extraction COMPLETE!
Saving feature array...
Done!


(Obsolete) PHASE 4 - Extract facial landmarks from an image


In [ ]:
print("Extracting facial landmarks STARTED...")
# Input directory containing images with extracted facial features
input_directory_extraction = 'Face_Output/Face_Output_Feature_Extraction'
# Output directory for saving images with facial landmarks
output_directory_landmarks = 'Face_Output'
face.extract_facial_landmarks(
    input_directory_extraction, output_directory_landmarks)
print("Extracting facial landmarks COMPLETE!")

(Obsolete) PHASE 5 - Convert facial landmarks into feature vectors


In [ ]:
print("Converting facial landmarks to feature vectors STARTED...")

# Assuming you have extracted landmarks and stored them in 'features'
# and you want to save the feature vectors in 'output_dir'
output_dir = 'Face_Output/Face_Output_LFCV'
face.landmarks_to_features(features, output_dir=output_dir)

print("Converting facial landmarks to feature vectors COMPLETE!")

# View the stored feature vectors
stored_feature_vectors = []

# Iterate over files in the output directory
for filename in os.listdir(output_dir):
    if filename.endswith('.npy'):  # Check if the file is a NumPy binary file
        # Construct the file path
        filepath = os.path.join(output_dir, filename)
        # Load the data from the file
        feature_vector = face.np.load(filepath)
        # Append the feature vector to the list
        stored_feature_vectors.append(feature_vector)
        # Print the contents of the feature vector
        print("Feature vector from file:", filename)
        print(feature_vector)

# Print the number of stored feature vectors
print("Number of feature vectors generated:", len(stored_feature_vectors))